In [1]:
# 로컬 벡터DB로부터 검색하고 LLM에 리트리버로 적용한다. (RAG)

In [1]:
import os
import warnings
from dotenv import load_dotenv

# 크로마와 FAISS 가 같이 설치되어있는 컴퓨터에선 아래 환경변수 설정을 해줘야 오류가 발생하지 않음
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings('ignore')

load_dotenv('./../.env')

True

In [2]:
# 동일한 임베딩 모델로 검색해야만 한다
from langchain_ollama import OllamaEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

ollama_url = 'http://localhost:11434'               # Ollama 설치 필수
embed_model = 'nomic-embed-text'                    # ollama pull nomic-embed-text (설치 필수)
embeddings = OllamaEmbeddings(model= embed_model, base_url= ollama_url)

db_path = './health_supplements'
vector_store = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)

In [3]:
### Retreival (벡터DB에서 유사도 검색)
knn_count = 5
question = "how to gain muscle mass?"
docs = vector_store.search(query=question, k=knn_count, search_type="similarity")

docs

[Document(id='8bad2d12-2318-401d-bb4e-dc36b513dc78', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset\\gym supplements\\2. High Prevalence of Supplement Intake.pdf', 'file_path': 'rag-dataset\\gym supplements\\2. High Prevalence of Supplement Intake.pdf', 'total_pages': 11, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:39:04+00:00', 'trapped': '', 'page': 8}, page_content='and strength gain among men. We detected more prevalent protein and creatine supplementation\namong younger compared to older ﬁtness center users, whereas the opposite was found for vitamin\nsupplementation. Other authors made similar observations [23] and it might reﬂect the diﬀerent\ntraining goals among age groups, i.e., more focus on strength and muscles among the younger and\nmore focus on health among the older age groups.\nComparable to other studies [4], we detected a positive correlation between training fre

In [4]:
# 리트리버 사용
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':3})
question = "how to gain muscle mass?"
retriever.invoke(question)

[Document(id='8bad2d12-2318-401d-bb4e-dc36b513dc78', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset\\gym supplements\\2. High Prevalence of Supplement Intake.pdf', 'file_path': 'rag-dataset\\gym supplements\\2. High Prevalence of Supplement Intake.pdf', 'total_pages': 11, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:39:04+00:00', 'trapped': '', 'page': 8}, page_content='and strength gain among men. We detected more prevalent protein and creatine supplementation\namong younger compared to older ﬁtness center users, whereas the opposite was found for vitamin\nsupplementation. Other authors made similar observations [23] and it might reﬂect the diﬀerent\ntraining goals among age groups, i.e., more focus on strength and muscles among the younger and\nmore focus on health among the older age groups.\nComparable to other studies [4], we detected a positive correlation between training fre

In [5]:
question = "how to lose weight?"
retriever.invoke(question)

[Document(id='38ac6032-4c8e-4958-a956-85e848ca67c1', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'file_path': 'rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'total_pages': 17, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:37:54+00:00', 'trapped': '', 'page': 12}, page_content='Int. J. Environ. Res. Public Health 2021, 18, 8897\n13 of 24\n3.3. Dietary Supplements and Weight Loss\nThe number of people with excessive weight continues to rise, and ﬁghting obesity\nhas become one of the greatest challenges of contemporary medicine. A person wishing to\nlose weight needs to undertake several difﬁcult life-changes and practice them consistently\n(diet, physical activity, addiction-free). Meanwhile dietary supplements are presented as\na compelling alternative to traditional methods for combatting obes

In [6]:
# 검색유형 : similarity_score_threshold
retriever = vector_store.as_retriever(search_type='similarity_score_threshold',
                                      search_kwargs={'k':3, 'score_threshold': 0.1})
question = "how to lose weight?"
retriever.invoke(question)

[Document(id='38ac6032-4c8e-4958-a956-85e848ca67c1', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'file_path': 'rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'total_pages': 17, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:37:54+00:00', 'trapped': '', 'page': 12}, page_content='Int. J. Environ. Res. Public Health 2021, 18, 8897\n13 of 24\n3.3. Dietary Supplements and Weight Loss\nThe number of people with excessive weight continues to rise, and ﬁghting obesity\nhas become one of the greatest challenges of contemporary medicine. A person wishing to\nlose weight needs to undertake several difﬁcult life-changes and practice them consistently\n(diet, physical activity, addiction-free). Meanwhile dietary supplements are presented as\na compelling alternative to traditional methods for combatting obes

In [7]:
# 검색유형 : mmr
retriever = vector_store.as_retriever(search_type='mmr',
                                      search_kwargs={'k':3, 'fetch_k': 20, 'lambda_mult':1})

# 20개를 먼저 유사도 검색하고 그 다음 유사도검색으로 3개를 선정
question = "how to lose weight?"
docs = retriever.invoke(question)
docs

[Document(id='38ac6032-4c8e-4958-a956-85e848ca67c1', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'file_path': 'rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'total_pages': 17, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:37:54+00:00', 'trapped': '', 'page': 12}, page_content='Int. J. Environ. Res. Public Health 2021, 18, 8897\n13 of 24\n3.3. Dietary Supplements and Weight Loss\nThe number of people with excessive weight continues to rise, and ﬁghting obesity\nhas become one of the greatest challenges of contemporary medicine. A person wishing to\nlose weight needs to undertake several difﬁcult life-changes and practice them consistently\n(diet, physical activity, addiction-free). Meanwhile dietary supplements are presented as\na compelling alternative to traditional methods for combatting obes

In [8]:
### RAG with LLAMA on OLLAMA
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from langchain import hub

In [9]:
# 프롬프트 작성
prompt = "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"

prompt = ChatPromptTemplate.from_template(prompt)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [10]:
# 다른 사람이 랭체인에 올려놓은 프롬프트를 가져다쓴다.
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [11]:
# LLM 셋팅
model_name = 'llama3.2:1b'
ollama_url = 'http://localhost:11434'

llm = ChatOllama(model=model_name, base_url=ollama_url)
llm

ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')

In [12]:
# 검색된 k개의 리트리버를 하나의 문자열로 만들기위한 함수
def format_docs(docs):
    return '\n\n'.join([doc.page_content for doc in docs])

In [13]:
# 리트리버들을 하나의 문자열로 합친다
context = format_docs(docs)
print(context)

Int. J. Environ. Res. Public Health 2021, 18, 8897
13 of 24
3.3. Dietary Supplements and Weight Loss
The number of people with excessive weight continues to rise, and ﬁghting obesity
has become one of the greatest challenges of contemporary medicine. A person wishing to
lose weight needs to undertake several difﬁcult life-changes and practice them consistently
(diet, physical activity, addiction-free). Meanwhile dietary supplements are presented as
a compelling alternative to traditional methods for combatting obesity. Wróbel-Harmas
et al. [107] demonstrated that weight loss supplements are the most frequently sought di-
etary supplements on the Internet, followed by preparations for muscle building and sexual
potency. In the US, more than 30% of people with overweight and obesity believe supple-
ments to be an effective method for losing weight [108], while in Poland these supplements
are used by as many as 40–50% of young women, regardless of their weight [15].

are used by as many a

In [14]:
# RAG 체인 생성
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
# rag_chain 을 통해 질문한다.
question = "how to lose weight?"
response = rag_chain.invoke(question)

print(response)

To lose weight, a combination of healthy diet and regular physical activity is recommended, but dietary supplements are not effective for this purpose. Consuming fat burners or taking multiple products at once can increase the risk of side effects, such as allergic reactions or serious health issues. It's essential to purchase products from authorized sellers and follow the recommended dosages to minimize risks.


In [16]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(target='ko')
response_ko = translator.translate(response)

print(response_ko)

체중 감량을 위해 건강한식이 요법과 규칙적인 신체 활동의 조합이 권장되지만식이 보조제는이 목적에 효과적이지 않습니다. 지방 버너를 소비하거나 한 번에 여러 제품을 복용하면 알레르기 반응 또는 심각한 건강 문제와 같은 부작용의 위험이 증가 할 수 있습니다. 공인 판매자로부터 제품을 구매하고 권장 복용량을 따라 위험을 최소화하는 것이 중요합니다.


In [17]:
question = "how to gain muscle mass?"
response = rag_chain.invoke(question)

print(response)

To gain muscle mass, focus on a well-balanced diet with sufficient protein and creatine supplementation, as well as regular training for two goals (improving health or body shaping). Adequate acid intake from supplements like vitamins can also contribute to muscle growth. However, individual results may vary due to weak correlation between supplement usage and muscle gain.


In [18]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(target='ko')
response_ko = translator.translate(response)

print(response_ko)

근육량을 얻으려면 충분한 단백질과 크레아틴 보충제로 균형 잡힌식이 요법과 두 가지 목표 (건강 또는 신체 형성 향상)를위한 정기적 인 훈련에 중점을 둡니다. 비타민과 같은 보충제로부터의 적절한 산 섭취는 또한 근육 성장에 기여할 수 있습니다. 그러나 보충제 사용량과 근육 이득 사이의 상관 관계가 약화되어 개별 결과가 달라질 수 있습니다.
